In [1]:
import numpy as np
import os
import time

In [2]:
import keras

In [3]:
keras.__version__

'2.4.3'

In [4]:
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [5]:
#!pip install -q -U tensorflow-addons==0.11.2

### Utilities

In [6]:
from utils import verifyDir
from utils.networks import normalize, unnormalize, plot_data

### Dataset

In [7]:
from utils.CIFAR10 import load_real_samples

### Discriminator & Generator

In [8]:
from utils.CIFAR10 import define_discriminator
from utils.CIFAR10 import define_generator

### Semi-Supervised GAN

In [9]:
from utils.networks import define_gan

### Selecting sub-set 

In [10]:
from utils.networks import select_supervised_samples, generate_real_samples
from utils.networks import generate_fake_samples, generate_latent_points

### Training

In [11]:
# train the generator and discriminator
def train(generator_model, unsupervised_model, supervised_model, gan_model, dataset, dataset_test, 
          latent_dim=100, n_epochs=20, n_batch=100, percent_samples=1.0):
    
    # select supervised dataset
    X_sup, y_sup = select_supervised_samples(dataset, percent_samples=percent_samples)
    print("Sup samples:", X_sup.shape, y_sup.shape)
    
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    print('n_epochs=%d, n_batch=%d, batch/epoch=%d, steps=%d' % (n_epochs, n_batch, bat_per_epo, n_steps))
    
    # manually enumerate epochs
    f_history = open(f"{LOG_PATH}SSL_GAN.csv", "w")
    f_history.write("step,generator_loss,unsupervised_real_loss,unsupervised_fake_loss,supervised_loss,supervised_acc,train_loss,test_loss,train_acc,test_acc\n")
    
    #for epoch in n_epochs:
    #    for batch in range(bat_per_epo):
    for step in range(1,n_steps+1):
#         t_start = time.time()
        # update supervised discriminator (c)
        [Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], n_batch)
        c_loss, c_acc = supervised_model.train_on_batch(Xsup_real, ysup_real)
        
        # update unsupervised discriminator (d)
        [X_real, _], y_real = generate_real_samples(dataset, n_batch)
        d_loss1, real_acc = unsupervised_model.train_on_batch(X_real, y_real)
        
        X_fake, y_fake = generate_fake_samples(generator_model, latent_dim, n_batch)
        d_loss2, fake_acc = unsupervised_model.train_on_batch(X_fake, y_fake)
        
        # update generator (g)
        X_gan, y_gan = generate_latent_points(latent_dim, n_batch), np.ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
#         t_total = (time.time() - t_start)
        # summarize loss on this batch
    
        # Train - Test
        X_train, y_train = dataset
        loss_train, acc_train = supervised_model.evaluate(X_train, y_train, verbose=0)

        # evaluate the test classifier model
        X_test, y_test = dataset_test
        loss_test, acc_test = supervised_model.evaluate(X_test, y_test, verbose=0)
        
        # Log
        print('epoch: %d | step: %d | Train: G_Loss: %.3f, ' \
              'D_unsup_loss_real: %.3f, D_unsup_acc_real:  %.2f, ' \
              'D_unsup_loss_fake: %.3f, D_unsup_acc_fake: %.2f, ' \
              'D_sup_loss: %.3f, D_sup_acc: %.2f ' \
              'Train acc: %.3f Test acc: %.3f ' %(int(step/bat_per_epo), step, g_loss,
                                                d_loss1, real_acc*100,
                                                d_loss2, fake_acc*100,
                                                c_loss, c_acc*100,
                                                 acc_train*100, acc_test*100))#, end = '\r')
        f_history.write(f"{step},{g_loss},{d_loss1},{d_loss2},{c_loss},{c_acc*100},{loss_train},{loss_test},{acc_train*100},{acc_test*100}\n")
        
        if step==1:
            plot_data(unnormalize(X_test).astype(int), 0, "test", grid_size = [10, 10], OUT_PATH=LOG_PATH)
        # evaluate the model performance every so often
        if (step) % (100) == 0 or step == 1:
            #summarize_performance(step, generator_model, supervised_model, latent_dim, dataset, dataset_test)
            # prepare fake examples
            X_generated, _ = generate_fake_samples(generator_model, latent_dim, n_samples=100)
            # scale from [-1,1] to [0,255]
            plot_data(unnormalize(X_generated).astype(int), step, "generated", grid_size = [10, 10], OUT_PATH=LOG_PATH)
            
            X_train, y_train = dataset
            _, acc = supervised_model.evaluate(X_train, y_train, verbose=0)
            print('Train Classifier Accuracy: %.3f%%\n' % (acc * 100))
            
            # evaluate the test classifier model
            X_test, y_test = dataset_test
            _, acc = supervised_model.evaluate(X_test, y_test, verbose=0)
            print('Test Classifier Accuracy: %.3f%%\n' % (acc * 100))
            # save the generator model
            filename2 = f'{LOG_PATH}generator_model_{step}.h5'
            generator_model.save(filename2)
            # save the classifier model
            filename3 = f'{LOG_PATH}supervised_model_{step}.h5'
            supervised_model.save(filename3)
            print('>Saving models Generator: %s and Supervised: %s' % (filename2, filename3))
    
    f_history.close()

### Parameters

In [12]:
input_shape = (32, 32, 3)
num_classes = 10

learning_rate =  0.0002

labeled_rate = 1.0
labeled_samples = 50000*labeled_rate

In [13]:
LOG_PATH = f"Logs/SSGAN_CIFAR10/Classifier_{int(labeled_samples)}/"
verifyDir(LOG_PATH)

### Creating models

In [14]:
# create the discriminator models
unsupervised_model, supervised_model = define_discriminator(in_shape=input_shape, n_classes=num_classes, learning_rate = learning_rate)
# create the generator
generator_model = define_generator(latent_dim=100)

In [15]:
supervised_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         147584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 128)         0     

In [16]:
unsupervised_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         147584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 128)         0   

In [17]:
generator_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              413696    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 4096)              0         
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 128)         524416    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 16, 16, 128)       2622

In [18]:
# create the gan
gan_model = define_gan(generator_model, unsupervised_model, learning_rate = learning_rate)

In [19]:
gan_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              413696    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 4096)              0         
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 128)         524416    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 16, 16, 128)       2622

### Loading Dataset

In [20]:
# load image data
dataset, dataset_test = load_real_samples()

(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)


### Training

In [ ]:
train(generator_model, unsupervised_model, supervised_model, gan_model, 
      dataset, dataset_test, latent_dim=100, 
      n_epochs=300, n_batch=128, percent_samples=labeled_rate)

Sup samples: (50000, 32, 32, 3) (50000,)
n_epochs=300, n_batch=128, batch/epoch=390, steps=117000
epoch: 0 | step: 1 | Train: G_Loss: 0.095, D_unsup_loss_real: 0.094, D_unsup_acc_real:  100.00, D_unsup_loss_fake: 2.400, D_unsup_acc_fake: 0.00, D_sup_loss: 2.312, D_sup_acc: 7.81 Train acc: 10.798 Test acc: 10.570 
Train Classifier Accuracy: 10.798%

Test Classifier Accuracy: 10.570%

>Saving models Generator: Logs/SSGAN_CIFAR10/Classifier_50000/generator_model_1.h5 and Supervised: Logs/SSGAN_CIFAR10/Classifier_50000/supervised_model_1.h5
epoch: 0 | step: 2 | Train: G_Loss: 0.095, D_unsup_loss_real: 0.091, D_unsup_acc_real:  100.00, D_unsup_loss_fake: 2.399, D_unsup_acc_fake: 0.00, D_sup_loss: 2.301, D_sup_acc: 10.54 Train acc: 13.200 Test acc: 13.400 
epoch: 0 | step: 3 | Train: G_Loss: 0.096, D_unsup_loss_real: 0.088, D_unsup_acc_real:  100.00, D_unsup_loss_fake: 2.398, D_unsup_acc_fake: 0.00, D_sup_loss: 2.294, D_sup_acc: 13.42 Train acc: 14.500 Test acc: 14.190 
epoch: 0 | step: 4 | 

epoch: 0 | step: 37 | Train: G_Loss: 8.009, D_unsup_loss_real: 0.418, D_unsup_acc_real:  85.16, D_unsup_loss_fake: 0.003, D_unsup_acc_fake: 100.00, D_sup_loss: 2.034, D_sup_acc: 28.24 Train acc: 26.538 Test acc: 27.190 
epoch: 0 | step: 38 | Train: G_Loss: 9.048, D_unsup_loss_real: 0.431, D_unsup_acc_real:  85.94, D_unsup_loss_fake: 0.487, D_unsup_acc_fake: 75.78, D_sup_loss: 2.010, D_sup_acc: 27.09 Train acc: 27.200 Test acc: 27.850 
epoch: 0 | step: 39 | Train: G_Loss: 7.645, D_unsup_loss_real: 0.488, D_unsup_acc_real:  82.03, D_unsup_loss_fake: 0.004, D_unsup_acc_fake: 100.00, D_sup_loss: 1.994, D_sup_acc: 27.80 Train acc: 27.622 Test acc: 28.250 
epoch: 0 | step: 40 | Train: G_Loss: 13.886, D_unsup_loss_real: 0.402, D_unsup_acc_real:  86.72, D_unsup_loss_fake: 0.511, D_unsup_acc_fake: 73.44, D_sup_loss: 1.983, D_sup_acc: 28.21 Train acc: 27.174 Test acc: 27.610 
epoch: 0 | step: 41 | Train: G_Loss: 13.396, D_unsup_loss_real: 0.717, D_unsup_acc_real:  79.69, D_unsup_loss_fake: 0.000

epoch: 0 | step: 75 | Train: G_Loss: 2.923, D_unsup_loss_real: 0.500, D_unsup_acc_real:  75.00, D_unsup_loss_fake: 0.191, D_unsup_acc_fake: 100.00, D_sup_loss: 1.925, D_sup_acc: 31.46 Train acc: 31.562 Test acc: 32.090 
epoch: 0 | step: 76 | Train: G_Loss: 3.450, D_unsup_loss_real: 0.207, D_unsup_acc_real:  91.41, D_unsup_loss_fake: 0.198, D_unsup_acc_fake: 100.00, D_sup_loss: 1.906, D_sup_acc: 32.07 Train acc: 31.144 Test acc: 32.100 
epoch: 0 | step: 77 | Train: G_Loss: 3.616, D_unsup_loss_real: 0.380, D_unsup_acc_real:  80.47, D_unsup_loss_fake: 0.165, D_unsup_acc_fake: 100.00, D_sup_loss: 1.911, D_sup_acc: 32.00 Train acc: 31.258 Test acc: 31.970 
epoch: 0 | step: 78 | Train: G_Loss: 3.318, D_unsup_loss_real: 0.443, D_unsup_acc_real:  71.09, D_unsup_loss_fake: 0.108, D_unsup_acc_fake: 100.00, D_sup_loss: 1.902, D_sup_acc: 31.90 Train acc: 31.700 Test acc: 32.380 
epoch: 0 | step: 79 | Train: G_Loss: 3.380, D_unsup_loss_real: 0.242, D_unsup_acc_real:  92.19, D_unsup_loss_fake: 0.138

epoch: 0 | step: 112 | Train: G_Loss: 3.036, D_unsup_loss_real: 0.436, D_unsup_acc_real:  78.12, D_unsup_loss_fake: 0.195, D_unsup_acc_fake: 100.00, D_sup_loss: 1.793, D_sup_acc: 35.81 Train acc: 35.478 Test acc: 36.470 
epoch: 0 | step: 113 | Train: G_Loss: 3.287, D_unsup_loss_real: 0.210, D_unsup_acc_real:  86.72, D_unsup_loss_fake: 0.180, D_unsup_acc_fake: 100.00, D_sup_loss: 1.788, D_sup_acc: 36.41 Train acc: 36.304 Test acc: 36.850 
epoch: 0 | step: 114 | Train: G_Loss: 3.549, D_unsup_loss_real: 0.261, D_unsup_acc_real:  85.94, D_unsup_loss_fake: 0.216, D_unsup_acc_fake: 100.00, D_sup_loss: 1.785, D_sup_acc: 36.79 Train acc: 36.380 Test acc: 36.900 
epoch: 0 | step: 115 | Train: G_Loss: 3.350, D_unsup_loss_real: 0.215, D_unsup_acc_real:  89.84, D_unsup_loss_fake: 0.090, D_unsup_acc_fake: 100.00, D_sup_loss: 1.787, D_sup_acc: 36.86 Train acc: 36.570 Test acc: 37.050 
epoch: 0 | step: 116 | Train: G_Loss: 3.077, D_unsup_loss_real: 0.340, D_unsup_acc_real:  85.16, D_unsup_loss_fake: 

epoch: 0 | step: 150 | Train: G_Loss: 3.082, D_unsup_loss_real: 0.151, D_unsup_acc_real:  94.53, D_unsup_loss_fake: 0.157, D_unsup_acc_fake: 98.44, D_sup_loss: 1.721, D_sup_acc: 39.12 Train acc: 38.730 Test acc: 39.020 
epoch: 0 | step: 151 | Train: G_Loss: 3.430, D_unsup_loss_real: 0.153, D_unsup_acc_real:  95.31, D_unsup_loss_fake: 0.430, D_unsup_acc_fake: 81.25, D_sup_loss: 1.716, D_sup_acc: 39.00 Train acc: 38.216 Test acc: 38.260 
epoch: 0 | step: 152 | Train: G_Loss: 3.393, D_unsup_loss_real: 0.357, D_unsup_acc_real:  78.91, D_unsup_loss_fake: 0.283, D_unsup_acc_fake: 90.62, D_sup_loss: 1.725, D_sup_acc: 38.24 Train acc: 37.928 Test acc: 37.910 
epoch: 0 | step: 153 | Train: G_Loss: 4.004, D_unsup_loss_real: 0.196, D_unsup_acc_real:  91.41, D_unsup_loss_fake: 0.533, D_unsup_acc_fake: 75.78, D_sup_loss: 1.725, D_sup_acc: 37.93 Train acc: 38.968 Test acc: 39.060 
epoch: 0 | step: 154 | Train: G_Loss: 3.222, D_unsup_loss_real: 0.290, D_unsup_acc_real:  85.16, D_unsup_loss_fake: 0.17

...

epoch: 2 | step: 1166 | Train: G_Loss: 2.391, D_unsup_loss_real: 0.628, D_unsup_acc_real:  67.19, D_unsup_loss_fake: 0.479, D_unsup_acc_fake: 85.16, D_sup_loss: 1.281, D_sup_acc: 53.88 Train acc: 55.698 Test acc: 54.060 
epoch: 2 | step: 1167 | Train: G_Loss: 1.982, D_unsup_loss_real: 0.828, D_unsup_acc_real:  59.38, D_unsup_loss_fake: 0.413, D_unsup_acc_fake: 83.59, D_sup_loss: 1.274, D_sup_acc: 54.08 Train acc: 55.632 Test acc: 54.300 
epoch: 2 | step: 1168 | Train: G_Loss: 2.281, D_unsup_loss_real: 0.681, D_unsup_acc_real:  60.16, D_unsup_loss_fake: 0.596, D_unsup_acc_fake: 64.84, D_sup_loss: 1.275, D_sup_acc: 54.32 Train acc: 55.818 Test acc: 54.380 
epoch: 2 | step: 1169 | Train: G_Loss: 2.439, D_unsup_loss_real: 0.717, D_unsup_acc_real:  60.94, D_unsup_loss_fake: 0.474, D_unsup_acc_fake: 82.03, D_sup_loss: 1.272, D_sup_acc: 54.50 Train acc: 55.888 Test acc: 54.070 
epoch: 3 | step: 1170 | Train: G_Loss: 2.373, D_unsup_loss_real: 0.506, D_unsup_acc_real:  71.88, D_unsup_loss_fake:

epoch: 3 | step: 1203 | Train: G_Loss: 1.829, D_unsup_loss_real: 0.525, D_unsup_acc_real:  69.53, D_unsup_loss_fake: 0.337, D_unsup_acc_fake: 92.19, D_sup_loss: 1.267, D_sup_acc: 54.54 Train acc: 55.954 Test acc: 53.720 
epoch: 3 | step: 1204 | Train: G_Loss: 1.721, D_unsup_loss_real: 0.461, D_unsup_acc_real:  71.09, D_unsup_loss_fake: 0.424, D_unsup_acc_fake: 85.94, D_sup_loss: 1.275, D_sup_acc: 53.70 Train acc: 55.572 Test acc: 53.390 
epoch: 3 | step: 1205 | Train: G_Loss: 1.856, D_unsup_loss_real: 0.413, D_unsup_acc_real:  74.22, D_unsup_loss_fake: 0.292, D_unsup_acc_fake: 93.75, D_sup_loss: 1.285, D_sup_acc: 53.32 Train acc: 55.960 Test acc: 53.810 
epoch: 3 | step: 1206 | Train: G_Loss: 1.714, D_unsup_loss_real: 0.611, D_unsup_acc_real:  63.28, D_unsup_loss_fake: 0.343, D_unsup_acc_fake: 93.75, D_sup_loss: 1.274, D_sup_acc: 53.90 Train acc: 56.714 Test acc: 54.570 
epoch: 3 | step: 1207 | Train: G_Loss: 1.773, D_unsup_loss_real: 0.517, D_unsup_acc_real:  67.19, D_unsup_loss_fake:

epoch: 3 | step: 1241 | Train: G_Loss: 1.579, D_unsup_loss_real: 0.584, D_unsup_acc_real:  71.88, D_unsup_loss_fake: 0.610, D_unsup_acc_fake: 71.09, D_sup_loss: 1.245, D_sup_acc: 55.65 Train acc: 57.556 Test acc: 55.820 
epoch: 3 | step: 1242 | Train: G_Loss: 1.837, D_unsup_loss_real: 0.475, D_unsup_acc_real:  69.53, D_unsup_loss_fake: 0.506, D_unsup_acc_fake: 79.69, D_sup_loss: 1.249, D_sup_acc: 55.88 Train acc: 57.044 Test acc: 55.100 
epoch: 3 | step: 1243 | Train: G_Loss: 1.841, D_unsup_loss_real: 0.442, D_unsup_acc_real:  78.91, D_unsup_loss_fake: 0.557, D_unsup_acc_fake: 73.44, D_sup_loss: 1.255, D_sup_acc: 55.28 Train acc: 56.774 Test acc: 54.840 
epoch: 3 | step: 1244 | Train: G_Loss: 1.767, D_unsup_loss_real: 0.482, D_unsup_acc_real:  71.88, D_unsup_loss_fake: 0.435, D_unsup_acc_fake: 83.59, D_sup_loss: 1.264, D_sup_acc: 54.80 Train acc: 56.762 Test acc: 54.790 
epoch: 3 | step: 1245 | Train: G_Loss: 1.903, D_unsup_loss_real: 0.399, D_unsup_acc_real:  78.12, D_unsup_loss_fake:

epoch: 3 | step: 1279 | Train: G_Loss: 1.961, D_unsup_loss_real: 0.814, D_unsup_acc_real:  56.25, D_unsup_loss_fake: 0.490, D_unsup_acc_fake: 78.12, D_sup_loss: 1.269, D_sup_acc: 55.25 Train acc: 56.852 Test acc: 55.450 
epoch: 3 | step: 1280 | Train: G_Loss: 2.124, D_unsup_loss_real: 0.677, D_unsup_acc_real:  62.50, D_unsup_loss_fake: 0.501, D_unsup_acc_fake: 74.22, D_sup_loss: 1.263, D_sup_acc: 55.45 Train acc: 56.672 Test acc: 55.040 
epoch: 3 | step: 1281 | Train: G_Loss: 1.923, D_unsup_loss_real: 0.691, D_unsup_acc_real:  63.28, D_unsup_loss_fake: 0.445, D_unsup_acc_fake: 84.38, D_sup_loss: 1.266, D_sup_acc: 55.07 Train acc: 56.318 Test acc: 54.680 
epoch: 3 | step: 1282 | Train: G_Loss: 1.938, D_unsup_loss_real: 0.566, D_unsup_acc_real:  74.22, D_unsup_loss_fake: 0.377, D_unsup_acc_fake: 85.94, D_sup_loss: 1.274, D_sup_acc: 54.68 Train acc: 56.372 Test acc: 54.840 
epoch: 3 | step: 1283 | Train: G_Loss: 2.011, D_unsup_loss_real: 0.594, D_unsup_acc_real:  64.84, D_unsup_loss_fake:

epoch: 3 | step: 1316 | Train: G_Loss: 1.993, D_unsup_loss_real: 0.452, D_unsup_acc_real:  75.00, D_unsup_loss_fake: 0.431, D_unsup_acc_fake: 82.81, D_sup_loss: 1.243, D_sup_acc: 55.82 Train acc: 58.176 Test acc: 56.290 
epoch: 3 | step: 1317 | Train: G_Loss: 2.097, D_unsup_loss_real: 0.366, D_unsup_acc_real:  81.25, D_unsup_loss_fake: 0.441, D_unsup_acc_fake: 84.38, D_sup_loss: 1.230, D_sup_acc: 56.33 Train acc: 58.128 Test acc: 56.350 
epoch: 3 | step: 1318 | Train: G_Loss: 1.948, D_unsup_loss_real: 0.463, D_unsup_acc_real:  75.00, D_unsup_loss_fake: 0.321, D_unsup_acc_fake: 92.97, D_sup_loss: 1.236, D_sup_acc: 56.40 Train acc: 57.892 Test acc: 56.170 
epoch: 3 | step: 1319 | Train: G_Loss: 1.681, D_unsup_loss_real: 0.383, D_unsup_acc_real:  78.91, D_unsup_loss_fake: 0.534, D_unsup_acc_fake: 74.22, D_sup_loss: 1.240, D_sup_acc: 56.15 Train acc: 57.994 Test acc: 56.300 
epoch: 3 | step: 1320 | Train: G_Loss: 1.784, D_unsup_loss_real: 0.392, D_unsup_acc_real:  82.03, D_unsup_loss_fake:

### Testing

In [ ]:
dataset, dataset_test = load_real_samples()

In [ ]:
X_train, y_train = dataset
_, acc = supervised_model.evaluate(X_train, y_train, verbose=0)
print('Train Classifier Accuracy: %.3f%%\n' % (acc * 100))

In [ ]:
X_test, y_test = dataset_test
_, acc = supervised_model.evaluate(X_test, y_test, verbose=0)
print('Test Classifier Accuracy: %.3f%%\n' % (acc * 100))

### Plotting

In [ ]:
import pandas as pd

In [ ]:
results_file = pd.read_csv(f"{LOG_PATH}SSL_GAN.csv")

In [ ]:
log_file = results_file.iloc[:,1:]
log_file

In [ ]:
log_file.iloc[:, [0,1,2,3]].plot(figsize=(12,8))

In [ ]:
log_file.iloc[:, [5,6]].plot(figsize=(12,8))

In [ ]:
log_file.iloc[:, [7,8]].plot(figsize=(12,8), ylim=(0,100), yticks=range(0,110,10))